In [8]:
from large_gridworld_ipythonblocks import BlockGrid
import numpy as np
import time
from IPython import display
import matplotlib.pyplot as plt
import pandas as pd

class small_gridworld():
    
    def __init__(self,grid_size):
        # use either a small pre-staged grid world or big one with random hazards
        
        ### initialize grid, agent, obstacles, etc.,
        self.width = 5
        self.height = 4
        self.grid = BlockGrid(self.width,self.height, fill=(200, 200, 200))
        
        # decide on hazards and goal locations
        self.hazards = [[0,3],[1,3],[2,3]]  # impenetrable obstacle locations          
        self.goal = [0,4]     # goal block
        self.player = [0,0]   # initial location player
        
        # index states for Q matrix
        self.states = []
        for i in range(self.grid.height):
            for j in range(self.grid.width):
                block = [i,j]
                self.states.append(str(i) + str(j))
        
        if grid_size == 'large':
            ### initialize grid, agent, obstacles, etc.,
            self.width = 20
            self.height = 10
            self.grid = BlockGrid(self.width,self.height, fill=(200, 200, 200))

            # decide on player and goal locations
            self.goal = [0,self.width-1]     # goal block
            self.player = [0,0]   # initial location player
            
            # index states for Q matrix
            self.states = []
            for i in range(self.grid.height):
                for j in range(self.grid.width):
                    block = [i,j]
                    self.states.append(str(i) + str(j))

            # decide on hazard locations
            num_hazards = 50
            self.hazards = []
            inds = np.random.permutation(self.grid.width*self.grid.height)
            inds = inds[:num_hazards]
            k = 0
            for i in range(self.grid.height):
                for j in range(self.grid.width):
                    if k in inds: 
                        block = [i,j]
                        if block != self.goal:
                            self.hazards.append(block)
                    k+=1
                                
        # initialize action choices
        self.action_choices = [[-1,0],[1,0],[0,-1],[0,1]]
        
        # initialize Q^* matrix
        self.Q_star = np.zeros((self.grid.width*self.grid.height,len(self.action_choices)))
        
    def color_grid(self):                            
        # remake + recolor grid
        self.grid = BlockGrid(self.width,self.height, fill=(200, 200, 200))

        # color obstacles
        for i in range(len(self.hazards)):
            self.grid[self.hazards[i][0],self.hazards[i][1]].green = 100
            self.grid[self.hazards[i][0],self.hazards[i][1]].red = 250
            self.grid[self.hazards[i][0],self.hazards[i][1]].blue = 0

        # make and color goal
        self.grid[self.goal[0],self.goal[1]].green = 255
        self.grid[self.goal[0],self.goal[1]].red = 0
        self.grid[self.goal[0],self.goal[1]].blue = 0
        
        # color player location
        self.grid[self.player[0],self.player[1]].green = 0
        self.grid[self.player[0],self.player[1]].red = 0
        self.grid[self.player[0],self.player[1]].blue = 200
        
        self.grid.show()
        
    ## Q-learning function
    def qlearn(self,gamma,hazard_penalty,num_train_animate):
        num_episodes = 300
        num_complete = 0
        
        # loop over episodes, for each run simulation and update Q
        for n in range(num_episodes):
            # pick random initialization 
            obstical_free = 0
            loc = [np.random.randint(self.grid.height),np.random.randint(self.grid.width)]
           
            # update Q matrix while loc != goal
            steps = 0  # step counter
            max_steps = 10*self.grid.width*self.grid.height  # maximum number of steps per episode
            while steps < max_steps:    
                # when you reach the goal end current episode
                if loc == self.goal:
                    break
                    
                ### choose action - left = 0, right = 1, up = 2, down = 3
                k = np.random.randint(len(self.action_choices))  
                loc2 = [sum(x) for x in zip(loc, self.action_choices[k])] 
                ind_old = self.states.index(str(loc[0]) + str(loc[1]))

                ### set reward    
                # is the new location just a regular square?  Than small negative reward
                r_k = int(-1)

                # if new state is hazard penalize with medium negative value - this needs to be set properly if you're trying to prove a point i.e., that a trained agent won't walk over one of these unless going around costs more
                if loc2 in self.hazards:
                    r_k = int(hazard_penalty)

                # if new state is goal set reward of 0
                if loc2 == self.goal:
                    r_k = int(0)
                    
                # if new state is outside of boundaries of grid world penalize set reward to small negative value (like -1) and do not move
                if loc2[0] > self.grid.height-1 or loc2[0] < 0 or loc2[1] > self.grid.width-1 or loc2[1] < 0:  
                    r_k = int(-1)
                    loc2 = loc

                ind_new = self.states.index(str(loc2[0]) + str(loc2[1]))
                self.Q_star[ind_old,k] = r_k + gamma*max(self.Q_star[ind_new,:])
                    
                # update current location of player to one we just moved too (or stay still if grid world boundary met)
                self.player = loc2
                loc = loc2
                
                # the next few lines just animate the first few steps during the first few episodes
                if n < num_train_animate and steps < 100:
                    self.color_grid()
                    time.sleep(0.1)
                    display.clear_output(wait=True)
                    
                # update counter
                steps+=1
        
            # pause briefly between first few episodes to show user cutoff
            if n < num_train_animate:
                time.sleep(1)
                print 'finished episode, animating next episode'
                time.sleep(1)
            else:
                display.clear_output(wait=True)
                print 'continuing with remaining episodes without animation...'
                
        print 'q-learning process complete'
                
    # print out
    def show_qmat(self):        
        df = pd.DataFrame(self.Q_star,columns=['up','down','left','right'], index=self.states)
        print df.round(3)             
            
    # animate the testing phase based on completed Q-learning cycle
    def animate_testing_phase(self,loc):
        # show movement based on an initial 
        self.player = loc # initial agent location

        # if you chose an invalid starting position, break out and try again
        if  loc == self.goal or loc[0] > self.grid.height-1 or loc[0] < 0 or loc[1] > self.grid.width-1 or loc[1] < 0:
            print 'initialization is outside of gridworld or is goal, try again'
        else:  
            # now use the learned Q* matrix to run from any (valid) initial point to the goal
            self.color_grid()
            time.sleep(0.3)
            display.clear_output(wait=True)
            count = 0
            max_count = self.grid.width*self.grid.height
            while count < max_count:
                # find next state using max Q* value
                ind_old = self.states.index(str(self.player[0]) + str(self.player[1]))
                
                # find biggest value in Q* and determine block location
                action_ind = np.argmax(self.Q_star[ind_old,:])
                action = self.action_choices[action_ind]
                
                # move player to new location and recolor - if you move out of gridworld halt and report this to user
                new_loc = [sum(x) for x in zip(self.player,action)] 
                
                if new_loc[0] > self.grid.height-1 or new_loc[0] < 0 or new_loc[1] > self.grid.width-1 or new_loc[1] < 0:
                    print 'something went wrong - the player has left the gridworld arena'
                    print "your episodes did not the states reached by your initialization enough to train Q properly here"
                    print "this is likely because you didn't traiin long enough - up the number of steps / episodes and try again"
                    
                self.player = new_loc
                
                # clear current screen for next step
                self.color_grid()
                time.sleep(0.2)
                if self.player == self.goal:
                    break
                display.clear_output(wait=True)
                count+=1

In [18]:
# create instance of grid world
test = small_gridworld(grid_size = 'large')

# train q learning and lets examine the matrix as it develops
test.qlearn(gamma = 0.8,hazard_penalty = -4,num_train_animate = 2)

continuing with remaining episodes without animation...
q-learning process complete


In [15]:
# run testing phase
starting_location = [0,0]
test.animate_testing_phase(starting_location)

,,,,
,,,,
,,,,
,,,,


In [17]:
test.show_qmat()

       up   down   left  right
00 -4.463 -4.329 -4.463 -4.329
01 -4.329 -4.161 -4.463 -4.161
02 -4.161 -3.951 -4.329 -4.000
03 -1.000 -4.800 -4.161  0.000
04  0.000  0.000  0.000  0.000
10 -4.463 -4.161 -4.329 -4.161
11 -4.329 -3.951 -4.329 -3.951
12 -4.161 -3.689 -4.161 -4.800
13 -4.000 -5.440 -3.951 -1.000
14  0.000 -1.800 -4.800 -1.000
20 -4.329 -3.951 -4.161 -3.951
21 -4.161 -3.689 -4.161 -3.689
22 -3.951 -3.362 -3.951 -5.440
23 -4.800 -2.952 -3.689 -1.800
24 -1.000 -2.440 -5.440 -1.800
30 -4.161 -3.951 -3.951 -3.689
31 -3.951 -3.689 -3.951 -3.362
32 -3.689 -3.362 -3.689 -2.952
33 -5.440 -2.952 -3.362 -2.440
34 -1.800 -2.440 -2.952 -2.440


In [1]:
# cart pole problem

In [18]:
import gym
env = gym.make('CartPole-v0')
for i_episode in range(1):
    observation = env.reset()
    for t in range(1):
#         env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

[2016-12-02 07:48:09,018] Making new env: CartPole-v0


[-0.01113717  0.00537443  0.01484656 -0.02708079]


In [21]:
# always begin at the same state
observation = env.reset()

Actions

0 - move to the right

1 - move to the left

In [39]:
observation

array([-0.03252785, -0.03166994, -0.02746844, -0.00841382])

This includes: 
- the cart position -> the 1st entry of the state

- the cart velocity -> the 2nd entry of the state

- the angle of the pole measured as its deviation from the vertical position - 3rd entry of state (in RADIANS)

- and the angular velocity of the pole - 4th entry of state

In [4]:
import gym
env = gym.make('CartPole-v0')
print(env.action_space)                # 0,1
print(env.observation_space)           # 4 continuous values
print(env.observation_space.high)
print(env.observation_space.low)

[2016-12-02 07:18:25,508] Making new env: CartPole-v0


Discrete(2)
Box(4,)
[  4.80000000e+00   3.40282347e+38   4.18879020e-01   3.40282347e+38]
[ -4.80000000e+00  -3.40282347e+38  -4.18879020e-01  -3.40282347e+38]


In [6]:
action = env.action_space.sample()
observation, reward, done, info = env.step(action)

In [45]:
import numpy as np

In [50]:
np.shape(env.observation_space)[0]

4

In [59]:
np.shape(observation)

(4,)

In [60]:
np.dot(observation,observation)

0.0028863538205226077

In [41]:
print(env.observation_space) 

Box(4,)


In [292]:
import gym
import numpy as np

class my_cartpole():

    def __init__(self):
        # initialize enviroment
        self.env = gym.make('CartPole-v0')
                                
        # initialize action choices
        self.action_choices = [0,1]
        
        # initialize Q matrix parameters
        self.Q = np.zeros((np.shape(env.observation_space)[0]+1,env.action_space.n))
        
    ## Q-learning function
    def qlearn(self,gamma):
        num_episodes = 200
        num_complete = 0
        
        # loop over episodes, for each run simulation and update Q
        for n in range(num_episodes):
            # we use the same initialization every time
            observation = env.reset()  # note here 'observation' = state
            
            # discretize, i.e., round observation features
            observation = np.around(observation,3)

            '''
             Note here that there are four features to the given state:
            - the cart position -> the 1st entry of the state
            - the cart velocity -> the 2nd entry of the state
            - the angle of the pole measured as its deviation from the vertical position - 3rd entry of state (in radians)
            - the angular velocity of the pole - 4th entry of state
            '''
            
            # start episode of simulation training
            steps = 0
            max_steps = 500
            while steps < max_steps:
                # take random action - either a 0 or 1
                action = env.action_space.sample()
                observation, reward, done, info = env.step(action)  # reward = +1 for every time unit the pole is above a threshold angle, 0 else

                # discretize, i.e., round observation features
                observation = np.around(observation,3)
                
                # compute long term reward based on proper linear model from Q
                state = np.insert(observation,0,1)
                h_i = np.dot(self.Q[:,action],state)
                
                # compute Q cost value
                q_k = reward + gamma*h_i
                
                # solve linear system to update paramters
                state.shape = (len(state),1)
                theta_i = np.linalg.pinv(state)*q_k
                self.Q[:,action] = theta_i.ravel()
                
                # if pole goes below threshold angle restart - new episode
                if done:
                    break

                # update counter
                steps+=1
                
        print 'q-learning process complete'      
    
    def animate_testing_phase(self):
        # start up simulation episode
        observation = env.reset()  # note here 'observation' = state
        
        # discretize, i.e., round observation features
        observation = np.around(observation,3)
        
        # start testing phase
        steps = 0
        max_steps = 500
        while steps < max_steps:
            state = np.insert(observation,0,1)
            state.shape = (len(state),1)

            # pick best action based on input state
            h = np.dot(self.Q.T,state)
            action = int(np.argmax(h))
            
            # take action 
            observation, reward, done, info = env.step(action)
            
            # discretize, i.e., round observation features
            observation = np.around(observation,3)
        
            # render action in animation
            # env.render()
            
            # if pole goes below threshold then end simulation
            if done:
                print("lasted {} timesteps".format(steps))
                break
                
            steps+=1

In [293]:
test = my_cartpole()
test.qlearn(gamma=0.8)

[2016-12-02 12:38:19,376] Making new env: CartPole-v0


q-learning process complete


In [297]:
test.animate_testing_phase()

lasted 8 timesteps


In [298]:
test.Q

array([[ 0.95771043,  1.02567891],
       [ 0.1924998 ,  0.18257085],
       [ 0.9548373 ,  1.21953223],
       [-0.22410424, -0.20103307],
       [-1.58884161, -1.93340475]])

In [279]:
a = test.Q.copy()

In [285]:
np.around(a,2)

array([[ 3.12,  2.7 ],
       [ 0.23,  0.2 ],
       [-0.47,  0.11],
       [-0.67, -0.53],
       [-1.93, -2.28]])